In [267]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
data = yf.download(tickers = '^GSPC', start = '2018-01-01',end = '2022-12-9')
data.head(10)


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,2683.729980,2695.889893,2682.360107,2695.810059,2695.810059,3397430000
2018-01-03,2697.850098,2714.370117,2697.770020,2713.060059,2713.060059,3544030000
2018-01-04,2719.310059,2729.290039,2719.070068,2723.989990,2723.989990,3697340000
2018-01-05,2731.330078,2743.449951,2727.919922,2743.149902,2743.149902,3239280000
2018-01-08,2742.669922,2748.510010,2737.600098,2747.709961,2747.709961,3246160000
2018-01-09,2751.149902,2759.139893,2747.860107,2751.290039,2751.290039,3467460000
2018-01-10,2745.550049,2750.800049,2736.060059,2748.229980,2748.229980,3579900000
2018-01-11,2752.969971,2767.560059,2752.780029,2767.560059,2767.560059,3645690000
2018-01-12,2770.179932,2787.850098,2769.639893,2786.239990,2786.239990,3587220000


In [268]:
type(data)

pandas.core.frame.DataFrame

In [269]:
# Adding indicators
data['RSI']=ta.rsi(data.Close, length=15)
data['EMAF']=ta.ema(data.Close, length=20)
data['EMAM']=ta.ema(data.Close, length=100)
data['EMAS']=ta.ema(data.Close, length=150)
data["log(Open)"]=np.log(data["Open"])
#Target variable = log(price(t+1)-log(t))
data["Return"]=data["log(Open)"].shift(-1)-data["log(Open)"]
# data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]
data.dropna(inplace=True)
data.reset_index(inplace = True)
data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)
data.head(10)

,Open,High,Low,Adj Close,RSI,EMAF,EMAM,EMAS,log(Open),Return
0,2840.290039,2853.290039,2835.979980,2850.399902,64.711067,2810.775016,2756.057446,2731.602135,7.951661,0.005488
1,2855.919922,2863.429932,2855.919922,2858.449951,66.268617,2815.315486,2758.085021,2733.282239,7.957149,0.000305
2,2856.790039,2862.439941,2853.090088,2857.699951,65.977928,2819.352102,2760.057594,2734.930156,7.957454,0.000140
3,2857.189941,2862.479980,2851.979980,2853.580078,64.317443,2822.611909,2761.909524,2736.501678,7.957594,-0.006422
4,2838.899902,2842.199951,2825.810059,2833.280029,56.774115,2823.627920,2763.322801,2737.783510,7.951172,-0.001212
5,2835.459961,2843.399902,2819.879883,2821.929932,53.047096,2823.466207,2764.483338,2738.898032,7.949959,-0.002677
6,2827.879883,2843.110107,2826.580078,2839.959961,57.765925,2825.037041,2765.977925,2740.236601,7.947283,0.000025
7,2827.949951,2827.949951,2802.489990,2818.370117,51.168281,2824.402096,2767.015394,2741.271482,7.947307,0.001233
8,2831.439941,2850.489990,2831.439941,2840.689941,56.652174,2825.953319,2768.474296,2742.588283,7.948541,0.002427
9,2838.320068,2855.629883,2833.729980,2850.129883,58.751302,2828.255849,2770.091238,2744.012675,7.950968,0.005485


In [270]:
data_set = data

data_set.head(20)
#print(data_set.shape)
#print(data.shape)
#print(type(data_set))

,Open,High,Low,Adj Close,RSI,EMAF,EMAM,EMAS,log(Open),Return
0,2840.290039,2853.290039,2835.979980,2850.399902,64.711067,2810.775016,2756.057446,2731.602135,7.951661,0.005488
1,2855.919922,2863.429932,2855.919922,2858.449951,66.268617,2815.315486,2758.085021,2733.282239,7.957149,0.000305
2,2856.790039,2862.439941,2853.090088,2857.699951,65.977928,2819.352102,2760.057594,2734.930156,7.957454,0.000140
3,2857.189941,2862.479980,2851.979980,2853.580078,64.317443,2822.611909,2761.909524,2736.501678,7.957594,-0.006422
4,2838.899902,2842.199951,2825.810059,2833.280029,56.774115,2823.627920,2763.322801,2737.783510,7.951172,-0.001212
5,2835.459961,2843.399902,2819.879883,2821.929932,53.047096,2823.466207,2764.483338,2738.898032,7.949959,-0.002677
6,2827.879883,2843.110107,2826.580078,2839.959961,57.765925,2825.037041,2765.977925,2740.236601,7.947283,0.000025
7,2827.949951,2827.949951,2802.489990,2818.370117,51.168281,2824.402096,2767.015394,2741.271482,7.947307,0.001233
8,2831.439941,2850.489990,2831.439941,2840.689941,56.652174,2825.953319,2768.474296,2742.588283,7.948541,0.002427
9,2838.320068,2855.629883,2833.729980,2850.129883,58.751302,2828.255849,2770.091238,2744.012675,7.950968,0.005485


In [263]:
#Target column Categories
#y =[1 if data.Open[i]>data.Close[i] else 0 for i in range(0, len(data))]
#yi = [data.Open[i]-data.Close[i] for i in range(0, len(data))]
#print(yi)
#print(len(yi))

In [264]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
data_set_scaled = sc.fit_transform(data_set)
data_set_scaled.shape[0]

1094

In [266]:
data_set_scaled.shape[0]

1094

In [265]:
# multiple feature from data provided to the model
X = []
#print(data_set_scaled[0].size)
#data_set_scaled=data_set.values
backcandles = 20
print(data_set_scaled.shape[0])
# Indpendent variables
# deleted range(8)
#new columns to access = list
for j in [0,4,5,6,7]:#data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]):#backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X)
print(X.shape)
print(y)
print(y.shape)

1094


IndexError: list index out of range

In [ ]:
#also comprehensions for X
#X = np.array([data_set_scaled[i-backcandles:i,:4].copy() for i in range(backcandles,len(data_set_scaled))])
#print(X)
#print(X.shape)

In [ ]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)

lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
inputs = LSTM(150, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train, y=y_train, batch_size=15, epochs=40, shuffle=True, validation_split = 0.1)

In [ ]:
y_pred = model.predict(X_test)
#y_pred=np.where(y_pred > 0.43, 1,0)
for i in range(10):
    print(y_pred[i], y_test[i])

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(y_test, color = 'black', label = 'Test')
plt.plot(y_pred, color = 'green', label = 'pred')
plt.axhline(y = 0, color = 'r', linestyle = '-', label="Zero")
plt.legend()
plt.show()